Loading the dataset to perform 10-fold cross-validation evaluation

In [ ]:
from Utils.LoadDataset import Dataloader

dataset  = Dataloader(data_path = "./Custom Dataset/Raw Data",
                  labels_path = "./Custom Dataset/labels.csv", 
                  seperator = ",", test_ratio = 0.2, validation_ratio = 0.2,
                  classnames=['running', 'walking', 'sitting', 'lying'])

Creating the stratified folds for splitting the dataset into 10 folds.

In [ ]:
# Creating fold for training.

from sklearn.model_selection import KFold,StratifiedKFold
stratifiedKfold = StratifiedKFold(n_splits = 10, random_state = None, shuffle = True)

Creating dictionaries for original model and the tflite model to access the precision, recall and f1-score of every class in the dataset.

In [ ]:
from collections import namedtuple, defaultdict

model_metrics = defaultdict(dict)
tflite_model_metrics = defaultdict(dict)
metric_names = ["precision", "recall", "f1-score"]

for classname in dataset.classnames:
    for metric in metric_names:
        model_metrics[classname][metric] = 0.0
        tflite_model_metrics[classname][metric] = 0.0

perfroming 10-fold cross-validation

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import sklearn
from sklearn.model_selection import KFold,StratifiedKFold
from Utils.Optimization import OptimizeModel

fold_no = 0

tf.random.set_seed(1)

for train, test in stratifiedKfold.split(dataset.all.x, dataset.all.y):

    model = keras.Sequential()
    model.add(keras.Input(shape = (50, 1, 3), name = "accelartion"))
    model.add(keras.layers.Conv2D(filters = 14, kernel_size = (5, 1), padding = "valid", activation = "relu"))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Conv2D(filters = len(dataset.classnames), kernel_size = (5, 1), padding = "same", 
                                  activation = "relu"))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.MaxPool2D((46, 1)))
    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(len(dataset.classnames), use_bias=True))

    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[keras.metrics.SparseCategoricalAccuracy()])
    
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    model.fit(dataset.all.x[train], dataset.all.y[train],
                    batch_size=64,
                    epochs=100,
                    verbose=0)
    
    model_predictions = model.predict(dataset.all.x[test])
    model_labels = np.argmax(model_predictions, axis = -1)
    
    model_classification_report = sklearn.metrics.classification_report((dataset.all.y[test]), 
                                                                        model_labels,
                                                                        target_names = (dataset.classnames), 
                                                                        digits = 4, 
                                                                        output_dict = True)
    
    for activity_name, metric_scores in model_classification_report.items():
        
        if activity_name in dataset.classnames:       
            for metric, value in metric_scores.items():
                
                if metric in metric_names:             
                    model_metrics[activity_name][metric] += model_classification_report[activity_name][metric]
                else:
                    continue
        else:
            continue
    
    tflite_converted_model = OptimizeModel(model, dataset, quantize = True, assertion = True)
    tflite_model_predictions = tflite_converted_model.predict(dataset.all.x[test])
    tflite_model_labels = np.argmax(tflite_model_predictions, axis = -1)
    
    tflite_model_classification_report = sklearn.metrics.classification_report((dataset.all.y[test]), 
                                                                        tflite_model_labels,
                                                                        target_names = (dataset.classnames), 
                                                                        digits = 4, 
                                                                        output_dict = True)
    
    for activity_name, metric_scores in tflite_model_classification_report.items():
        
        if activity_name in dataset.classnames:       
            for metric, value in metric_scores.items():
                
                if metric in metric_names:             
                    tflite_model_metrics[activity_name][metric] += tflite_model_classification_report[activity_name][metric]
                else:
                    continue
        else:
            continue
    
    fold_no = fold_no + 1
    

The performance estimates of all the classes evaluated using 10-fold cross-validation are below,

In [ ]:
model_metrics

In [ ]:
tflite_model_metrics